In [77]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import datasets, linear_model

/Users/sumitbhanwala/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [37]:
# function for reading the data and returning the
# the data to the linear functions for processing the data
# we do preprocessing here itself 
def readSAP500():
    """Returns a dataframe with the results for Google and S&P 500"""
  
  # Point to where you've stored the CSV file on your local machine
    googFile = 'data/GOOG.csv'
    spFile = 'data/SP_500.csv'
    goog = pd.read_csv(googFile, sep=",", usecols=[0,5], names=['Date','Goog'], header=0)
    sp = pd.read_csv(spFile, sep=",", usecols=[0,5], names=['Date','SP500'], header=0)
    goog['SP500'] = sp['SP500']
  # The date object is a string, format it as a date
    goog['Date'] = pd.to_datetime(goog['Date'], format='%Y-%m-%d')
    goog = goog.sort_values(['Date'], ascending=[True])
    returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64', 'int64']]]\
            .pct_change()
    return returns
    

In [53]:
def getData():
    data = readSAP500()
    xData = np.array(data["SP500"])[1:]
    yData = np.array(data["Goog"])[1:]
    return (xData, yData);


In [75]:
def scikitLinear():
    xData,  yData = getData();
    ''' Creating a  regression Object from the sklean library which we will use on our data '''
    regressionObject = linear_model.LinearRegression();
    regressionObject.fit(xData.reshape(-1, 1), yData.reshape(-1, 1))
    print('Coefficients: \n', regressionObject.coef_)
    print("the intercept is: \n", regressionObject.intercept_)

In [76]:
scikitLinear()

Coefficients: 
 [[ 1.06768973]]
the intercept is: 
 [ 0.0084649]


In [95]:
def tensorFlowLinearRegression():
    xData,  yData = getData();
    ## W haas to be a matrix as we have our X in the 
    ## for of the matrix while b is a scalar
    W = tf.Variable(tf.zeros([1,1]) ,name = "W");
    b = tf.Variable( tf.zeros([1]) , name = 'b');
    X = tf.placeholder(tf.float32, [None , 1] ,name = 'X');
    yreal = tf.placeholder(tf.float32, [None , 1] ,name = 'yreal');
    matrixProduct = tf.matmul(X, W);
    Y = matrixProduct + b;
    cost = tf.reduce_mean(tf.square(Y- yreal));
    trainStep = tf.train.FtrlOptimizer(1).minimize(cost);
    init = tf.global_variables_initializer();
    with tf.Session() as sess:
        sess.run(init);
        feed = {X: xData.reshape(-1, 1), yreal: yData.reshape(-1, 1)}
        for i in range(10000):
            sess.run(trainStep , feed_dict = feed);
            if( i == 10000 -1):
                print(sess.run(W));
                print(sess.run(b));
        
    

In [96]:
tensorFlowLinearRegression();

[[ 1.06768572]]
[ 0.00846492]
